## Authors: Zhaocai Liu, Phoebe Ho
## Purpose: This notebook is used to process GTFS data for the RouteE-BEAT Tool

In [1]:
## GTFS reference: https://gtfs.org/schedule/reference/#tripstxt

# Load libraries

In [1]:
import pandas as pd
import geopandas as gpd
import time
import os 
import numpy as np

import sqlalchemy as sql
from mappymatch.constructs.coordinate import Coordinate
from mappymatch.constructs.geofence import Geofence
from mappymatch.constructs.trace import Trace

from nrel.mappymatch.readers.tomtom import read_tomtom_nxmap_from_sql
from nrel.mappymatch.readers.tomtom_config import TomTomConfig 
from mappymatch.matchers.lcss.lcss import LCSSMatcher
from gradeit.gradeit import gradeit
from shapely.geometry import Polygon, Point

# Load data

City variable: choose from "saltlake" or "richmond", or other agencies with data

In [2]:
# select city for analysis
#city = 'Niagara_Region_Transit'
city = 'richmond'#'RTD'#'saltlake'
import pandas as pd

# load data
df_shape = pd.read_csv(f'./GTFS_Data/{city}/shapes.txt', sep=',', header=0)
df_route = pd.read_csv(f'./GTFS_Data/{city}/routes.txt', sep=',', header=0)
df_trips = pd.read_csv(f'./GTFS_Data/{city}/trips.txt', sep=',', header=0)
df_stops = pd.read_csv(f'./GTFS_Data/{city}/stops.txt', sep=',', header=0)
df_stops_times = pd.read_csv(f'./GTFS_Data/{city}/stop_times.txt', sep=',', header=0)


In [3]:
# Check number of routes
df_route.shape

(38, 9)

In [4]:
# Show route_type; type 3 is transit buses
df_route.route_type.unique()

array([3])

### Select shapes for bus routes only

"route_type" column indicates the type of transportation used on a route. Valid options are:

0 - Tram, Streetcar, Light rail. Any light rail or street level system within a metropolitan area.

1 - Subway, Metro. Any underground rail system within a metropolitan area.

2 - Rail. Used for intercity or long-distance travel.

3 - Bus. Used for short- and long-distance bus routes.

4 - Ferry. Used for short- and long-distance boat service.

5 - Cable tram. Used for street-level rail cars where the cable runs beneath the vehicle (e.g., cable car in San Francisco).

6 - Aerial lift, suspended cable car (e.g., gondola lift, aerial tramway). Cable transport where cabins, cars, gondolas or open chairs are suspended by means of one or more cables.

7 - Funicular. Any rail system designed for steep inclines.

11 - Trolleybus. Electric buses that draw power from overhead wires using poles.

12 - Monorail. Railway in which the track consists of a single rail or a beam.

In [5]:
# Extract Bus Only Shapes
shape_route_key = df_trips[['shape_id','route_id']].drop_duplicates() 

df_shape_bus = pd.merge(df_shape, shape_route_key, how='left', on='shape_id')
df_shape_bus = pd.merge(df_shape_bus, df_route[['route_id','route_type']], how='left', on='route_id')

# Select only bus services (route_type = 3)
print("All Shapes: ", df_shape.shape)
df_shape_bus = df_shape_bus[df_shape_bus['route_type']==3]
df_shape_bus = df_shape_bus.drop(columns=['route_id', 'route_type'])
df_shape = df_shape_bus.copy()
print("Bus Only Shapes: ", df_shape.shape)

All Shapes:  (138941, 5)
Bus Only Shapes:  (137557, 5)


## GTFS Outputs for R Shiny Dashboard
- Unique shapes for each service block
- Unique stops for each shape in each service block

In [7]:
import os
if not os.path.exists('./R_Shiny_Results'):
    os.mkdir('./R_Shiny_Results')
if not os.path.exists('./R_Shiny_Results/{}'.format(city)):
    os.mkdir('./R_Shiny_Results/{}'.format(city))

In [8]:
# Let's store the block shapes and block stops for each day of the week
# Seems like some GTFS data do not have calendar data, we treat them as outliars for now

try:
    df_calendar = pd.read_csv(f'./GTFS_Data/{city}/calendar.txt', sep=',', header=0)
except:
    print("The df_calendar file for {} read in failed !!!".format(city))


In [9]:
# We will need to rely on df_calendar to understand the service_id for each day of the week.

# We assume that only one start_date and one end_date (seem like a lot of transit agencies do not meet this requirement)
# Instead we ensure that the min end date is larger than max start date such that all service blocks have overlaps at least

### A dictionary to save the list of service_id for each day of the week
dict_service_id_list = {}
if len(df_calendar.start_date.unique()) == 1 and len(df_calendar.end_date.unique()) == 1:
    print("Unique Start and End Date!")
    
    for day_of_week in ['monday', 'tuesday', 'wednesday', 'thursday', 'friday','saturday', 'sunday']:
        if day_of_week in df_calendar.columns:
            dict_service_id_list[day_of_week] = df_calendar[df_calendar[day_of_week] == 1].service_id.to_list()
        else: ### Deal with the case where column name has an additional space
            match_column = [col_tmp for col_tmp in df_calendar.columns if day_of_week in col_tmp][0]
            dict_service_id_list[day_of_week] = df_calendar[df_calendar[match_column] == 1].service_id.to_list()

elif df_calendar.start_date.max() < df_calendar.end_date.min(): ### Make sure overlap exist for all service block effective dates
    print("The df_calendar file for {} is abnormal!!!".format(city),"but the dates have overlapped region!")
    
    for day_of_week in ['monday', 'tuesday', 'wednesday', 'thursday', 'friday','saturday', 'sunday']:
        if day_of_week in df_calendar.columns:
            dict_service_id_list[day_of_week] = df_calendar[df_calendar[day_of_week] == 1].service_id.to_list()
        else: ### Deal with the case where column name has an additional space
            match_column = [col_tmp for col_tmp in df_calendar.columns if day_of_week in col_tmp][0]
            dict_service_id_list[day_of_week] = df_calendar[df_calendar[match_column] == 1].service_id.to_list()

### Remove those special day (one day services)
else:
    print("Warning!!!, The df_calendar file for {} is abnormal!!!".format(city),"but the dates have no overlapped region!")
    ### Only consider the largest (normal range)

    df_calendar['start_date'] = df_calendar['start_date'].apply(lambda x: str(x)[0:4] + '-' + str(x)[4:6] + '-' + str(x)[6:])
    df_calendar['end_date'] = df_calendar['end_date'].apply(lambda x: str(x)[0:4] + '-' + str(x)[4:6] + '-' + str(x)[6:])

    df_calendar['start_date'] = pd.to_datetime(df_calendar['start_date'])
    df_calendar['end_date'] = pd.to_datetime(df_calendar['end_date'])

    df_calendar['service_duration'] = df_calendar['end_date']  - df_calendar['start_date'] 
    df_calendar = df_calendar[df_calendar.service_duration == df_calendar.service_duration.max()]

    for day_of_week in ['monday', 'tuesday', 'wednesday', 'thursday', 'friday','saturday', 'sunday']:
        if day_of_week in df_calendar.columns:
            dict_service_id_list[day_of_week] = df_calendar[df_calendar[day_of_week] == 1].service_id.to_list()
        else: ### Deal with the case where column name has an additional space
            match_column = [col_tmp for col_tmp in df_calendar.columns if day_of_week in col_tmp][0]
            dict_service_id_list[day_of_week] = df_calendar[df_calendar[match_column] == 1].service_id.to_list()

Unique Start and End Date!


In [10]:
dict_service_id_list

{'monday': [1],
 'tuesday': [1],
 'wednesday': [1],
 'thursday': [1],
 'friday': [1],
 'saturday': [3],
 'sunday': [2]}

In [11]:
## Merge trip shape with shape dataframe to have the 'shape_pt_lat', 'shape_pt_lon'

df_trips_block = df_trips[['route_id','block_id', 'shape_id','service_id']] #'trip_headsign','direction_id',
df_trips_block = df_trips_block.drop_duplicates(ignore_index=True)

df_trips_block = pd.merge(df_trips_block, df_route[['route_id','route_type','route_short_name']], how='left', on='route_id')

# select only bus services (route_type = 3)
print("All routes block shape: ", df_trips_block.shape)
df_trips_block_bus = df_trips_block[df_trips_block['route_type']==3]
print("Bus only reoute block shape: ", df_trips_block_bus.shape)



All routes block shape:  (1265, 6)
Bus only reoute block shape:  (1265, 6)


In [12]:
### Further merge with shape to get the gps points

block_shapes = pd.merge(df_trips_block_bus, df_shape, how='left', on='shape_id')
select_cols = ['route_id', 'block_id', 'shape_id','route_short_name',  # 'trip_headsign', 'direction_id', 
       'shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence','service_id']

print('All Blocks:',len(block_shapes.block_id.unique()))
block_shapes = block_shapes[select_cols].dropna() ##Drop nan to filter out blocks without bus shapes (i.e., rail etc.)
print('Transit Bus Blocks:',len(block_shapes.block_id.unique()))
block_shapes.to_csv('./R_Shiny_Results/{}/block_shapes_{}.csv'.format(city, city), index=False)

All Blocks: 349
Transit Bus Blocks: 349


In [13]:
###  Now extract stops
df_stops_merged = pd.merge(df_stops_times, df_stops, how='left', on='stop_id')

df_stops_merged = df_stops_merged[['trip_id','stop_id', 'stop_sequence',
                                  'stop_name', 'stop_desc', 'stop_lat','stop_lon']]
df_block_stops = pd.merge(df_trips, df_stops_merged, how='left', on='trip_id')
# remove trip_id, just want shapes and stops for each block
df_block_stops = df_block_stops[['route_id','direction_id', 'block_id', 'shape_id', 
                                'stop_id', 'stop_sequence', 'stop_name', 'stop_desc',
                                 'stop_lat', 'stop_lon']]

### Merge route to get route name
df_block_stops  = df_block_stops.merge(df_route[['route_id','route_short_name']], how='left', on='route_id')

print('All Blocks:',len(df_block_stops.block_id.unique()))
df_block_stops = df_block_stops[df_block_stops.block_id.isin(block_shapes.block_id.unique())] ## Only get stop for transit buses

print('Transit Bus Blocks:',len(df_block_stops.block_id.unique()))
df_block_stops = df_block_stops.drop_duplicates(ignore_index=True)

df_block_stops.to_csv('./R_Shiny_Results/{}/block_stops_{}.csv'.format(city, city), index=False)

All Blocks: 349
Transit Bus Blocks: 349


In [14]:
### For each block_id merge the route short name
block_shapes_group = block_shapes[['block_id','route_short_name']].groupby('block_id').agg(lambda x: 'Route_' + '_Route_'.join(x.unique()))

block_shapes_group = block_shapes_group.reset_index()

block_shapes_group['block_id_new'] =block_shapes_group['route_short_name'] + '_' +  block_shapes_group['block_id'].astype(str)

In [15]:
### Extract the blocks and shapes for each day of the week

for day_of_week in ['monday', 'tuesday', 'wednesday', 'thursday', 'friday','saturday', 'sunday']:
    service_block_list = dict_service_id_list[day_of_week]
    print(len(block_shapes[block_shapes.service_id.isin(service_block_list)].block_id.unique()), " Blocks for ",day_of_week)
    block_shapes_tmp_day = block_shapes[block_shapes.service_id.isin(service_block_list)]
    df_block_stops_tmp_day = df_block_stops[df_block_stops.block_id.isin(block_shapes_tmp_day.block_id.unique())]


    block_shapes_tmp_day = block_shapes_tmp_day.merge(block_shapes_group[['block_id','block_id_new']],how = 'left',on='block_id')

    block_shapes_tmp_day['block_id'] = block_shapes_tmp_day['block_id_new']
    block_shapes_tmp_day.to_csv('./R_Shiny_Results/{}/block_shapes_{}_{}.csv'.format(city, city, day_of_week), index=False)

    df_block_stops_tmp_day = df_block_stops_tmp_day.merge(block_shapes_group[['block_id','block_id_new']],how = 'left',on='block_id')
    df_block_stops_tmp_day['block_id'] = df_block_stops_tmp_day['block_id_new']
    df_block_stops_tmp_day.to_csv('./R_Shiny_Results/{}/block_stops_{}_{}.csv'.format(city, city, day_of_week), index=False)

110  Blocks for  monday
110  Blocks for  tuesday
110  Blocks for  wednesday
110  Blocks for  thursday
110  Blocks for  friday
81  Blocks for  saturday
63  Blocks for  sunday


In [16]:
# We might further compare those number of service blocks with the fleet size
# to see whether it is feasible to serve each block with one vehicle.